In [ ]:
import numpy as np
import time
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import qiskit as q
import qiskit.tools.jupyter
from qiskit.pulse import pulse_lib as _  # only want to call pulse_lib as q.pulse.pulse_lib

In [ ]:
import pulse_compiler_helper_fns

In [ ]:
q.IBMQ.load_account() # Load account from disk

In [ ]:
provider = q.IBMQ.get_provider(group='qiskit-terra', project='pulse-testing')
backend = provider.get_backend('alt_almaden')
system = q.pulse.PulseChannelSpec.from_backend(backend)
cmd_def = q.pulse.CmdDef.from_defaults(backend.defaults().cmd_def, backend.defaults().pulse_library)
measure = cmd_def.get('measure', qubits=backend.configuration().meas_map[0])

In [ ]:
thetas = np.linspace(0, np.pi/2, 20, endpoint=False)
print(thetas)

-------
## Standard Way: Cnot RZ Cnot

In [ ]:
schedules = []
for theta in thetas:
    circuit = q.QuantumCircuit(2)
    circuit.h(0)
    circuit.h(1)
    circuit.cx(0, 1)
    circuit.rx(theta, 1)
    circuit.cx(0, 1)
    circuit.h(0)
    circuit.h(1)

    decomposed_circuit = circuit.decompose()

    pulse_compiler_helper_fns.update_basis_gates_and_cmd_def(decomposed_circuit, backend, system, cmd_def)
    transpiled_circuit = q.transpile(decomposed_circuit, backend, optimization_level=1)

    schedule = q.schedule(transpiled_circuit, backend=backend, cmd_def=cmd_def)
    schedule |= measure << schedule.duration
    schedules.append(schedule)

In [ ]:
shots = 1000
standard_job = q.execute(schedules, backend=backend, shots=shots)
print('job is %s' % standard_job.job_id())

-------
## Optimized Way Way: zz_interaction decomposes directly to cross resonance

In [ ]:
schedules = []
for theta in thetas:
    circuit = q.QuantumCircuit(2)
    circuit.h(0)
    circuit.h(1)
    circuit.zz_interaction(theta, 0, 1)
    circuit.h(0)
    circuit.h(1)

    decomposed_circuit = circuit.decompose()

    pulse_compiler_helper_fns.update_basis_gates_and_cmd_def(decomposed_circuit, backend, system, cmd_def)
    transpiled_circuit = q.transpile(decomposed_circuit, backend, optimization_level=1)

    schedule = q.schedule(transpiled_circuit, backend=backend, cmd_def=cmd_def)
    schedule |= measure << schedule.duration
    schedules.append(schedule)

In [ ]:
shots = 1000
optimized_job = q.execute(schedules, backend=backend, shots=shots)
print('job is %s' % optimized_job.job_id())

-----
## Analysis

In [ ]:
from qiskit.ignis.verification.tomography.data import marginal_counts

In [ ]:
result = standard_job.result()
standard_counts_list = []
for i in range(len(thetas)):
    counts = marginal_counts(result.get_counts(i), meas_qubits=[0,1])    
    standard_counts_list.append(counts)
print(standard_counts_list)

In [ ]:
result = optimized_job.result()
optimized_counts_list = []
for i in range(len(thetas)):
    counts = marginal_counts(result.get_counts(i), meas_qubits=[0,1])    
    optimized_counts_list.append(counts)
print(optimized_counts_list)

In [ ]:
# # infer output state vectors, up to phase of each element (phase won't matter for overlap calculation)
# standard_state_vectors = []
# for counts in standard_counts_list:
#         standard_state_vectors.append(np.array([
#             np.sqrt(counts.get('00', 0)/shots),
#             np.sqrt(counts.get('01', 0)/shots),
#             np.sqrt(counts.get('10', 0)/shots),
#             np.sqrt(counts.get('11', 0)/shots)]))

In [ ]:
# optimized_state_vectors = []
# for counts in standard_counts_list:
#         standard_state_vectors.append(np.array([
#             np.sqrt(counts.get('00', 0)/shots),
#             np.sqrt(counts.get('01', 0)/shots),
#             np.sqrt(counts.get('10', 0)/shots),
#             np.sqrt(counts.get('11', 0)/shots)]))

In [ ]:
# pranav can add fidelity plotting cells below here